<a href="https://colab.research.google.com/github/Palakbhatt/fake_news/blob/main/fake_news_by_palak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

In [2]:
from google.colab import files
uploaded=files.upload()

Saving news.csv to news.csv


In [3]:
df = pd.read_csv('news.csv')

In [4]:
df.head(10)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [6]:
df.describe()

,Unnamed: 0
count,6335.000000
mean,5280.415627
std,3038.503953
min,2.000000
25%,2674.500000
50%,5271.000000
75%,7901.000000
max,10557.000000


In [7]:
df=df.sample(frac=1)
df.head(10)

,Unnamed: 0,title,text,label
1102,3856,"Obama Spends Earth Day In The Everglades, Taun...","""2014 was the planet's warmest year on record....",REAL
1016,7587,Show biz: Business and breakthroughs,Show biz: Business and breakthroughs Exclusive...,FAKE
6237,7836,Shock Claim: Nibiru Will Hit In 2017- World Go...,The head of a leading survivalist group has ma...,FAKE
5988,6189,Soy should not be consumed in significant quan...,Soy should not be consumed in significant quan...,FAKE
3030,6117,"Trump Haters Mow Down Signs, “Black Women for ...",A Twitter page known as Black Women for Trump ...,FAKE
3474,6500,"CNN Fired Donna Brazile 2 Weeks Ago, Made It P...",\n\nCNN finally made it public that they have ...,FAKE
4618,152,"Supreme Court, Trump engulf Capitol Hill",Notable names include Ray Washburne (Commerce)...,REAL
3767,8490,BREAKING: Police Raid Democratic Office – Hill...,"0 comments \nOn Tuesday, the Pennsylvania Stat...",FAKE
4601,2847,Antiquities destroyed at Palmyra by the Islami...,"A general view taken on March 27, 2016, shows ...",REAL
4460,107,Charleston exposes ugliest truth of our time: ...,"In the days after King’s assassination, Americ...",REAL


In [8]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [9]:
def word_drop(text):
  text=text.lower()
  text=re.sub('\[.*?\]', '', text)
  text=re.sub('\\W'," ",text)
  text=re.sub('https?://\S+\|www\.\S+', '', text)
  text=re.sub('<.*?>+', '', text)
  text=re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text=re.sub('\n', '', text)
  text=re.sub('\w*\d\w*', '', text)
  return text

In [10]:
df["text"]=df["text"].apply(word_drop)

In [11]:
df.head(10)

,Unnamed: 0,title,text,label
1102,3856,"Obama Spends Earth Day In The Everglades, Taun...",was the planet s warmest year on record fou...,REAL
1016,7587,Show biz: Business and breakthroughs,show biz business and breakthroughs exclusive...,FAKE
6237,7836,Shock Claim: Nibiru Will Hit In 2017- World Go...,the head of a leading survivalist group has ma...,FAKE
5988,6189,Soy should not be consumed in significant quan...,soy should not be consumed in significant quan...,FAKE
3030,6117,"Trump Haters Mow Down Signs, “Black Women for ...",a twitter page known as black women for trump ...,FAKE
3474,6500,"CNN Fired Donna Brazile 2 Weeks Ago, Made It P...",cnn finally made it public that they have se...,FAKE
4618,152,"Supreme Court, Trump engulf Capitol Hill",notable names include ray washburne commerce ...,REAL
3767,8490,BREAKING: Police Raid Democratic Office – Hill...,comments on tuesday the pennsylvania state ...,FAKE
4601,2847,Antiquities destroyed at Palmyra by the Islami...,a general view taken on march shows part o...,REAL
4460,107,Charleston exposes ugliest truth of our time: ...,in the days after king s assassination americ...,REAL


In [12]:
x=df['text']
y=df['label']

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

***LOGISTIC*** ***REGRESSION***

In [16]:
from sklearn.linear_model import LogisticRegression

In [17]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
LR.score(xv_test, y_test)

0.9040404040404041

In [19]:
pred_LR = LR.predict(xv_test)

In [20]:
print(classification_report(y_test, pred_LR))

              precision    recall  f1-score   support

        FAKE       0.89      0.92      0.91       801
        REAL       0.92      0.89      0.90       783

    accuracy                           0.90      1584
   macro avg       0.90      0.90      0.90      1584
weighted avg       0.90      0.90      0.90      1584



***DECISION TREE CLASSIFICATION***

In [21]:
from sklearn.tree import DecisionTreeClassifier

In [22]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [23]:
DT.score(xv_test, y_test)

0.7960858585858586

In [24]:
pred_DT = DT.predict(xv_test)

In [25]:
print(classification_report(y_test, pred_DT))

              precision    recall  f1-score   support

        FAKE       0.81      0.78      0.79       801
        REAL       0.78      0.81      0.80       783

    accuracy                           0.80      1584
   macro avg       0.80      0.80      0.80      1584
weighted avg       0.80      0.80      0.80      1584



In [26]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_DT)

array([[625, 176],
       [147, 636]])

In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred_DT)

0.7960858585858586

***GRADIENT BOOSTING CLASSIFIER***

In [28]:
from sklearn.ensemble import GradientBoostingClassifier

In [29]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [30]:
GBC.score(xv_test, y_test)

0.8977272727272727

In [31]:
pred_GBC = GBC.predict(xv_test)

In [32]:
print(classification_report(y_test, pred_GBC))

              precision    recall  f1-score   support

        FAKE       0.89      0.92      0.90       801
        REAL       0.91      0.88      0.89       783

    accuracy                           0.90      1584
   macro avg       0.90      0.90      0.90      1584
weighted avg       0.90      0.90      0.90      1584



In [33]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_GBC)

array([[733,  68],
       [ 94, 689]])

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred_DT)

0.7960858585858586

***RANDOM FOREST CLASSIFIER***

In [35]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [37]:
RFC.score(xv_test, y_test)

0.8952020202020202

In [38]:
pred_RFC = RFC.predict(xv_test)

In [39]:
print(classification_report(y_test, pred_RFC))

              precision    recall  f1-score   support

        FAKE       0.89      0.90      0.90       801
        REAL       0.90      0.89      0.89       783

    accuracy                           0.90      1584
   macro avg       0.90      0.90      0.90      1584
weighted avg       0.90      0.90      0.90      1584



In [40]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_RFC)

array([[722,  79],
       [ 87, 696]])

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred_RFC)

0.8952020202020202